In [ ]:
# для примера использую библиотеку neuralprophet
import pandas as pd
import numpy as np
from neuralprophet import NeuralProphet
import os
import pickle
from tqdm.auto import tqdm

И второй вариант, если модель строить только на данных аэропорта прилета, метрика RSME в этом случае
значительно лучше, менее 10% за счет большего объема данных

In [ ]:
airports = pd.read_csv('airports.csv')
fligshts = pd.read_csv('flights.csv')

In [ ]:
#объединяю данные, что бы в данных было найменование аэропорта и код аэропорта
#датасет с цифровой расшифровкой аэропортов не нашел, здесь так же удаляю эти данные

fligshts = fligshts.merge(airports, how='right', left_on='ORIGIN_AIRPORT', right_on='IATA_CODE')

In [ ]:
#добавляю колонку с датой в формате datetime64

fligshts['Date'] = fligshts[['YEAR','MONTH','DAY']].astype('str').agg('-'. join , axis= 1)
fligshts['Date'] = pd.to_datetime(fligshts['Date']).dt.date.astype('datetime64[ns]')

#удаляю колонки с датой, месяцем, годом

fligshts.drop(['YEAR', 'DAY', 'MONTH'], axis=1, inplace=True)

In [ ]:
fligshts = fligshts[['Date', 'ORIGIN_AIRPORT', 'DEPARTURE_DELAY', 'DESTINATION_AIRPORT', 'ARRIVAL_DELAY']].reset_index()
fligshts.dropna(inplace=True)

In [ ]:
fligshts['ARRIVAL_DELAY'] = fligshts['ARRIVAL_DELAY'].astype('int')
fligshts['DEPARTURE_DELAY'] = fligshts['DEPARTURE_DELAY'].astype('int')

In [ ]:
#здесь формирую колонку PROBABILITY - если самолет опоздал, то значение 0
# из опоздания прилета вычитаю задержку на вылете, что бы исключить вину в этой задержке принимающего аэропорта
#прилетел раньше или в срок - значение 1


fligshts['DELAY'] = fligshts['ARRIVAL_DELAY'] - fligshts['DEPARTURE_DELAY']
fligshts['PROBABILITY'] = np.where(fligshts['DELAY']>0, 0, 1)

In [ ]:
fligshts.groupby('PROBABILITY').count()

In [ ]:
DESTINATION_AIRPORT = list(fligshts.DESTINATION_AIRPORT.unique())

In [ ]:
for airport in tqdm(DESTINATION_AIRPORT):
    df = fligshts.query(f'DESTINATION_AIRPORT == "{airport}"') #датафрейм с конкретным аэропортом прилета
    B = df.groupby('Date')['PROBABILITY'].mean().reset_index() #фрейм с датой и средним значением вероятности задержки в этот аэропорт по дням
    B.columns = ['ds', 'y'] #привожу колонки к наименованию, которые необходимы для модели
    m = NeuralProphet() #создаем экземпляр класса
    metrics = m.fit(B, freq="D") #обучаем модель
    with open('{}'.format(airport)+'.pkl', 'wb') as f:
            pickle.dump(metrics, f)

#хочу оформить все в streamlit - нашел, как сделать пользовательский ввод, например пользователь вводит город вылета,
#далее формируется список аэропортов в этом городе, выбирается аэропотр, например, дата вылета

# выгружаю модели из соответствующей папки - в зависимости от выбранного аэропорта
# делаю предсказание, отвечаю на вопрос задания и сопутствующую визуализацию строю    